In [243]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

#### Построить рекомендации (регрессия, предсказываем оценку) на фичах:

Буду строить не регрессию, а классификацию, так как оценки это качественная переменная, а не количественная.  
Классификатор справится с этой задачей лучше. 

**логика A**:
- 3 фильма, которым уникальный для каждого пользователя классификатор присвоил рейтинг 5,  будут рекомендованы пользователю.

- Если пользователь поставил только одну оценку или все время ставил одну и ту же оценку, то:
    - будет использован метод наименьших соседей. 
    - Тот фильм, который находится ближе всех к просмотренным фильмам будет рекомендован пользователю. 

1. Соберем все таблицы
1. Соберем для пользователя все метрики
1. Соберем все метрики general (в общем). Чтобы бегать по ним при поиске лучшей рекомендации
1. Выберем классификатор который лучше всех справляется с задачей.
1. Соберем для каждого пользователя уникальный классификатор
1. Пробежим по всем метрикам для каждого пользователя и порекомендуем ему тот фильм согласно **логике A**
1. Средние оценки (+ median, variance, etc.) пользователя и фильма

PS. Оценить RMSE на тестовой выборке не проводится, так ка при выборе класификатора уже производится разбивка на train, test и записывается score на test. ( смотри **таблица с рекомендациями. Последняя строка 'Score'**)

#### Соберем все таблицы

In [ ]:
ratings_tags = ratings.join(tags.set_index(['userId','movieId' ]) ,on=['userId','movieId' ],rsuffix='r')

In [42]:
ratings_tags.drop(columns=['timestamp','timestampr'],inplace=True)

In [52]:
ratings_tags_movies = ratings_tags.join(movies.set_index('movieId'), on='movieId'  ).sort_values(by='userId')

In [54]:
for i in ratings_tags_movies.columns:
    print(i ,ratings_tags_movies[i].isna().sum() )
    

userId 0
movieId 0
rating 0
tag 99201
title 0
genres 0


Заменим пустые значения фразой 'there_is_empty_tag'

In [56]:
ratings_tags_movies.fillna(value='there_is_empty_tag',inplace=True)

In [57]:
for i in ratings_tags_movies.columns:
    print(i ,ratings_tags_movies[i].isna().sum() )
    

userId 0
movieId 0
rating 0
tag 0
title 0
genres 0


собранная таблица

In [58]:
ratings_tags_movies.head(3)

,userId,movieId,rating,tag,title,genres
0,1,1,4.0,there_is_empty_tag,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
147,1,2329,5.0,there_is_empty_tag,American History X (1998),Crime|Drama
148,1,2338,2.0,there_is_empty_tag,I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller


#### Соберем для пользователя все метрики

In [ ]:
tags_grouped = []
genres_grouped = []
movies_grouped = []
rating_grouped = []

for user_name,dataset in ratings_tags_movies.groupby(by='userId'):
    tags_grouped.append( dataset.tag.values)
    genres_grouped.append( [i.replace('|',' ') for i in dataset.genres.values]  )
    movies_grouped.append( dataset.movieId.values )
    rating_grouped.append( (dataset.rating.values*2).astype(np.int8) ) # Небольшая хитрость для того чтобы использовать Классификатор а не регрессию
    

#### Соберем все метрики general (в общем). Чтобы бегать по ним при поиске лучшей рекомендации

In [389]:
all_films_tags_grouped = []
all_films_genres_grouped = []
all_films_titles = []
all_films_rating_grouped = []

for movie_id,dataset in ratings_tags_movies.groupby(by='movieId'):
    all_films_tags_grouped.append( [' '.join(set( dataset.tag.values) ) ]  )
    all_films_genres_grouped.append( dataset.genres.iloc[0].replace('|',' ')   )
    all_films_titles.append( dataset.title.iloc[0])
    all_films_rating_grouped.append( dataset.rating.values )
    

#### Выберем классификатор который лучше всех справляется с задачей.

##### Сравнивал:
- Логит(multinominal,ovr)
- RandomForest  

Результаты одинаковые, Логит в 15 раз быстрее. Далее усовершенствовал логит

- bootstrap + Logit
- GridSearch + bootstrap + Logit

##### Победил Логит (multinominal) из коробки

In [251]:
logit = LogisticRegression(solver = 'saga', multi_class='multinomial')
X_train, X_test, y_train, y_test = train_test_split(X_genres_tags_sparse_tfidf.toarray(),rating_grouped[0],test_size=.15,random_state=42)

In [262]:
# Результаты логита
logit.fit(X_train,y_train)
logit.score(X_train,y_train), logit.score(X_test,y_test)

(0.5786802030456852, 0.6)

In [263]:
# результаты RandomForestClassifier 
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train,y_train)
rf.score(X_train,y_train), rf.score(X_test,y_test)

(0.7766497461928934, 0.4857142857142857)

#### Соберем для каждого пользователя уникальный классификатор

In [490]:
set_for_each_user = dict()

for i in range(len(genres_grouped)):
    genres_count_vect = CountVectorizer()
    tags_count_vect = CountVectorizer(ngram_range =(1,2)) # Это для тэгов разделять на одно и два слова, например "Том Харди"
    tfidf_transformer = TfidfTransformer()

    X_genres_sparse = genres_count_vect.fit_transform(genres_grouped[i])
    X_tags_sparse = tags_count_vect.fit_transform(tags_grouped[i])
    X_genres_tags_sparse = np.hstack(  ( X_genres_sparse.toarray(),X_tags_sparse.toarray() )  )  

    X_genres_tags_sparse_tfidf = tfidf_transformer.fit_transform(X_genres_tags_sparse)

    if len( set( rating_grouped[i] ) ) == 1: # пользователь всегда ставил одну и ту же оценку 
        neigh = NearestNeighbors(n_neighbors=5, n_jobs=-1, metric='manhattan') 
        neigh.fit(X_genres_tags_sparse_tfidf.toarray())
        algo = neigh
        score = 'knn'
    else:
        # среди классификаторов Победил логит 
        logit = LogisticRegression(solver = 'saga', multi_class='multinomial')
        # train test split
        X_train, X_test, y_train, y_test = train_test_split(X_genres_tags_sparse_tfidf.toarray(),rating_grouped[i],test_size=.05,random_state=42)

        logit.fit(X_train,y_train)
        algo = logit
        score = logit.score(X_test,y_test)

    set_for_each_user[i+1] = {'genres_coun_vectorizer':genres_count_vect,
                            'tags_count_vect':tags_count_vect,
                           'tf_idf_transformer':tfidf_transformer,
                           'algoritm': algo,
                           'algo_score':score }


#### Пробежим по всем метрикам для каждого пользователя и порекомендуем ему тот фильм согласно логике A

In [503]:
all_users = ratings_tags_movies.userId.unique()
def find_better_film(user,n_films=3):
    if user not in all_users: return print('Error. user id is not exist')
    results = []
    results_4 = []
    flag_4, flag_5 = False, False
    
    for i in range( len(all_films_titles)  ):
        film_genres_sparse = set_for_each_user[user]['genres_coun_vectorizer'].transform([all_films_genres_grouped[i]])
        film_tags_sparse = set_for_each_user[user]['tags_count_vect'].transform(all_films_tags_grouped[i] )
        film_genres_tags_sparse = np.hstack(  ( film_genres_sparse.toarray(),film_tags_sparse.toarray() )  )  

        film_genres_tags_sparse_tfidf = set_for_each_user[user]['tf_idf_transformer'].transform(film_genres_tags_sparse)
        
        if set_for_each_user[user]['algo_score'] != 'knn':
            y_pred = set_for_each_user[user]['algoritm'].predict(film_genres_tags_sparse_tfidf.toarray() )
            y_pred = int(y_pred/2)
        else: 
            flag_4 = set_for_each_user[user]['algoritm'].kneighbors(film_genres_tags_sparse_tfidf.toarray(), return_distance=True)[0].min()<.7
            flag_5 = set_for_each_user[user]['algoritm'].kneighbors(film_genres_tags_sparse_tfidf.toarray(), return_distance=True)[0].min()<.2
            if flag_5: y_pred = 5
            elif flag_4: y_pred = 4
            else:  y_pred = 0
            flag_4, flag_5 = False, False
        
        if  y_pred == 5: results.append(all_films_titles[i])
        if  y_pred == 4: results_4.append(all_films_titles[i])
        
        if len(results) == n_films: return results,set_for_each_user[user]['algo_score']
    
    if len(results) < n_films: 
        a = n_films - len(results)
        results.append(results_4[:a])
    return results,set_for_each_user[user]['algo_score']

In [505]:
results = []

for i in all_users:
        results.append(find_better_film(i))

#### Таблица рекомендаций:

In [514]:
columns = ['_'.join(['user',str(i)]) for i in all_users]

In [515]:
columns

['user_1',
 'user_2',
 'user_3',
 'user_4',
 'user_5',
 'user_6',
 'user_7',
 'user_8',
 'user_9',
 'user_10',
 'user_11',
 'user_12',
 'user_13',
 'user_14',
 'user_15',
 'user_16',
 'user_17',
 'user_18',
 'user_19',
 'user_20',
 'user_21',
 'user_22',
 'user_23',
 'user_24',
 'user_25',
 'user_26',
 'user_27',
 'user_28',
 'user_29',
 'user_30',
 'user_31',
 'user_32',
 'user_33',
 'user_34',
 'user_35',
 'user_36',
 'user_37',
 'user_38',
 'user_39',
 'user_40',
 'user_41',
 'user_42',
 'user_43',
 'user_44',
 'user_45',
 'user_46',
 'user_47',
 'user_48',
 'user_49',
 'user_50',
 'user_51',
 'user_52',
 'user_53',
 'user_54',
 'user_55',
 'user_56',
 'user_57',
 'user_58',
 'user_59',
 'user_60',
 'user_61',
 'user_62',
 'user_63',
 'user_64',
 'user_65',
 'user_66',
 'user_67',
 'user_68',
 'user_69',
 'user_70',
 'user_71',
 'user_72',
 'user_73',
 'user_74',
 'user_75',
 'user_76',
 'user_77',
 'user_78',
 'user_79',
 'user_80',
 'user_81',
 'user_82',
 'user_83',
 'user_84',
 

In [513]:
len(results)

610

In [524]:
final_table = pd.DataFrame(  data=results, index=columns, columns=['recommended_films','probability'] )
final_table.head(10)

,recommended_films,probability
user_1,"[Toy Story (1995), Jumanji (1995), Father of t...",0.333333
user_2,"[Toy Story (1995), Jumanji (1995), Grumpier Ol...",0
user_3,"[GoldenEye (1995), Twelve Monkeys (a.k.a. 12 M...",1
user_4,"[Toy Story (1995), Jumanji (1995), Father of t...",0.181818
user_5,"[Pocahontas (1995), Wild Bill (1995), Farinell...",0.333333
user_6,"[Lion King, The (1994), Aladdin (1992), Missio...",0.375
user_7,"[Species (1995), Mary Shelley's Frankenstein (...",0.125
user_8,"[Tom and Huck (1995), Balto (1995), Casino (19...",0.333333
user_9,"[Jumanji (1995), Cutthroat Island (1995), City...",0
user_10,"[Heat (1995), Sudden Death (1995), Casino (1995)]",0.428571


#### Средние оценки (+ median, variance, etc.) пользователя и фильма

##### По фильмам

In [58]:
ratings_tags_movies.head(3)

,userId,movieId,rating,tag,title,genres
0,1,1,4.0,there_is_empty_tag,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
147,1,2329,5.0,there_is_empty_tag,American History X (1998),Crime|Drama
148,1,2338,2.0,there_is_empty_tag,I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller


In [508]:
ratings_tags_movies.groupby(by='movieId').rating.agg(['mean', 'median','std'])

,mean,median,std
movieId,,,
1,3.920930,4.0,0.834859
2,3.441964,3.5,0.876997
3,3.245283,3.0,1.049829
4,2.357143,3.0,0.852168
5,3.040000,3.0,0.924938
...,...,...,...
193581,4.000000,4.0,NaN
193583,3.500000,3.5,NaN
193585,3.500000,3.5,NaN


##### По пользователям

In [58]:
ratings_tags_movies.head(3)

,userId,movieId,rating,tag,title,genres
0,1,1,4.0,there_is_empty_tag,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
147,1,2329,5.0,there_is_empty_tag,American History X (1998),Crime|Drama
148,1,2338,2.0,there_is_empty_tag,I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller


In [509]:
ratings_tags_movies.groupby(by='userId').rating.agg(['mean', 'median','std'])

,mean,median,std
userId,,,
1,4.366379,5.0,0.800048
2,4.128571,4.0,0.834397
3,2.435897,0.5,2.090642
4,3.555556,4.0,1.314204
5,3.636364,4.0,0.990441
...,...,...,...
606,3.657399,4.0,0.724121
607,3.786096,4.0,0.965657
608,3.134176,3.0,1.079262


In [510]:
ratings_tags_movies.groupby(by='userId').agg(std =('rating', np.std), median=('rating', np.median), mean=('rating', np.mean))

,std,median,mean
userId,,,
1,0.800048,5.0,4.366379
2,0.834397,4.0,4.128571
3,2.090642,0.5,2.435897
4,1.314204,4.0,3.555556
5,0.990441,4.0,3.636364
...,...,...,...
606,0.724121,4.0,3.657399
607,0.965657,4.0,3.786096
608,1.079262,3.0,3.134176
